# DATA PROCESSING AND CALCULATION OF xG-ADJUSTED FPL POINTS

## Setup and pre-processing

In [1]:
# give the number of the latest FPL round
latest_gameweek = 26

In [2]:
# import basic libraries
import pandas as pd
import numpy as np
import json
import requests
from scipy.stats import poisson

# allow more data columns to be shown than by default
pd.set_option('max_columns',100)

In [3]:
# import player data 
filepath = '../data/fbref/player_stats_week' + str(latest_gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)
playerStats.head()

,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,Gls,Ast,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK,G+A-PK,xG,npxG,xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA,Matches
Rk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Patrick van Aanholt\Patrick-van-Aanholt,nl NED,DF,Crystal Palace,28.0,1990.0,19,19,1624.0,2,0,1,1,0,0,0.11,0.00,0.11,0.06,0.06,1.7,0.9,0.9,0.09,0.05,0.14,0.05,0.10,Matches
2,Max Aarons\Max-Aarons,eng ENG,DF,Norwich City,19.0,2000.0,24,24,2160.0,0,0,0,0,5,0,0.00,0.00,0.00,0.00,0.00,0.4,0.4,3.4,0.02,0.14,0.16,0.02,0.16,Matches
3,Tammy Abraham\Tammy-Abraham,eng ENG,FW,Chelsea,21.0,1997.0,24,23,1926.0,13,3,0,0,2,0,0.61,0.14,0.75,0.61,0.75,11.7,11.7,2.3,0.55,0.11,0.65,0.55,0.65,Matches
4,Che Adams\Che-Adams,eng ENG,FW,Southampton,23.0,1996.0,20,8,695.0,0,1,0,0,0,0,0.00,0.13,0.13,0.00,0.13,1.5,1.5,0.6,0.20,0.08,0.28,0.20,0.28,Matches
5,Adrián\Adrian,es ESP,GK,Liverpool,32.0,1987.0,10,8,783.0,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches


In [4]:
# import team data and pre-process
filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col=0)#, engine='python')

# change team names to match convention used in the FPL data
teamStats.loc[teamStats['Squad']=='Brighton & Hove Albion','Squad'] = 'Brighton'
teamStats.loc[teamStats['Squad']=='Manchester United','Squad'] = 'Manchester Utd'
teamStats.loc[teamStats['Squad']=='Newcastle United','Squad'] = 'Newcastle Utd'
teamStats.loc[teamStats['Squad']=='Sheffield United','Squad'] = 'Sheffield Utd'
teamStats.loc[teamStats['Squad']=='West Ham United','Squad'] = 'West Ham'
teamStats.loc[teamStats['Squad']=='Tottenham Hotspur','Squad'] = 'Tottenham'
teamStats.loc[teamStats['Squad']=='Wolverhampton Wanderers','Squad'] = 'Wolves'

teamStats.head()

,Squad,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
Rk,,,,,,,,,,,,,,,,,,
1,Liverpool,26,25,1,0,61,15,46,76,51.9,25.1,26.8,1.03,W W W W W,53116,Mohamed Salah - 14,Alisson,NaN
2,Manchester City,26,17,3,6,67,29,38,54,62.2,23.6,38.5,1.48,W D W L W,54391,Sergio Agüero - 16,Ederson,NaN
3,Leicester City,26,15,5,6,54,26,28,50,42.3,31.6,10.7,0.41,L L W D D,32056,Jamie Vardy - 17,Kasper Schmeichel,NaN
4,Chelsea,26,12,5,9,43,36,7,41,44.7,25.1,19.6,0.75,W L D D L,40549,Tammy Abraham - 13,Kepa Arrizabalaga,NaN
5,Tottenham,26,11,7,8,43,34,9,40,35.2,36.1,-0.9,-0.04,L D W W W,59485,Harry Kane - 11,Paulo Gazzaniga,NaN


In [5]:
# fetch FPL data online
data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
df = pd.DataFrame(data['elements'])
df.set_index('id',inplace=True)

# fetch data locally
#df = pd.read_csv('../data/data_week' + str(latest_gameweek) + '.csv', index_col=0)#,encoding='latin-1')

df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,in_dreamteam,influence,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_saved,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2,1,125,100.0,100.0,2,69140,0,0,-4,4,14.6,0,2,6.2,6.2,6,Shkodran,5.7,7,0,26.7,False,148.2,530,,2020-01-27T22:30:15.833744Z,51,0,0,0,69140.jpg,3.6,0,0,Mustafi,0.4,False,None,a,1,3,103.0,25,12175,1606,35332,521,1.1,4.9,Mustafi,0
2,0,3,102,100.0,100.0,2,98745,0,0,-1,1,34.9,0,2,7.8,7.8,6,Héctor,7.3,8,1,18.0,False,109.8,532,,2019-12-09T20:00:21.228098Z,54,0,0,0,98745.jpg,3.8,0,0,Bellerín,0.9,False,None,a,1,3,34.0,23,94040,15378,46438,2658,1.4,4.3,Bellerín,2
3,2,1,219,100.0,50.0,2,111457,0,0,-3,3,156.3,0,2,0.5,0.2,0,Sead,0.0,18,0,41.2,False,190.0,1086,,2020-01-15T13:30:21.603499Z,52,0,0,0,111457.jpg,2.1,0,0,Kolasinac,0.5,False,None,a,1,3,67.0,34,55346,122,126152,339,0.0,6.5,Kolasinac,3
4,2,3,216,100.0,100.0,3,154043,0,0,-4,4,155.4,1,2,0.5,0.5,0,Ainsley,0.0,19,0,44.9,False,261.8,1210,,2019-09-22T18:00:10.824841Z,46,0,0,0,154043.jpg,2.6,1,0,Maitland-Niles,2.4,False,None,a,1,3,37.0,36,589573,391,617375,3960,0.0,7.8,Maitland-Niles,3
5,0,5,286,100.0,100.0,3,39476,0,0,-1,1,31.8,1,2,0.5,0.5,0,Sokratis,0.0,25,2,57.1,False,431.6,1606,,2020-01-18T17:00:22.046513Z,49,0,0,0,39476.jpg,2.9,0,0,Papastathopoulos,1.4,False,None,a,1,3,106.0,52,143786,441,186479,2225,0.0,10.6,Sokratis,5


## Probability to keep a clean sheet

Here, we estimate for each team the probability that the team keeps a clean sheet (against average opposition). We do this by first calculating the expected goals allowed per game for each team. Then, we assume that conceding goals follows a Poisson distribution, from which we then get the desired probability.

In [6]:
teamStats['xGA per game'] = teamStats['xGA'] / teamStats['MP']
teamStats['probability no goals allowed'] = poisson.pmf(0,teamStats['xGA per game'])
teamStats

,Squad,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xGA per game,probability no goals allowed
Rk,,,,,,,,,,,,,,,,,,,,
1,Liverpool,26,25,1,0,61,15,46,76,51.9,25.1,26.8,1.03,W W W W W,53116,Mohamed Salah - 14,Alisson,NaN,0.965385,0.380837
2,Manchester City,26,17,3,6,67,29,38,54,62.2,23.6,38.5,1.48,W D W L W,54391,Sergio Agüero - 16,Ederson,NaN,0.907692,0.403454
3,Leicester City,26,15,5,6,54,26,28,50,42.3,31.6,10.7,0.41,L L W D D,32056,Jamie Vardy - 17,Kasper Schmeichel,NaN,1.215385,0.296596
4,Chelsea,26,12,5,9,43,36,7,41,44.7,25.1,19.6,0.75,W L D D L,40549,Tammy Abraham - 13,Kepa Arrizabalaga,NaN,0.965385,0.380837
5,Tottenham,26,11,7,8,43,34,9,40,35.2,36.1,-0.9,-0.04,L D W W W,59485,Harry Kane - 11,Paulo Gazzaniga,NaN,1.388462,0.249459
6,Sheffield Utd,26,10,9,7,28,24,4,39,30.9,31.4,-0.5,-0.02,W D L W W,30751,John Fleck Lys Mousset - 5,Dean Henderson,NaN,1.207692,0.298886
7,Manchester Utd,26,10,8,8,38,29,9,38,39.4,26.1,13.3,0.51,W L L D W,72635,Marcus Rashford - 14,David de Gea,NaN,1.003846,0.366467
8,Wolves,26,8,12,6,35,32,3,36,33.0,25.7,7.3,0.28,D W L D D,31375,Raúl Jiménez - 11,Rui Patrício,NaN,0.988462,0.372149
9,Everton,26,10,6,10,34,38,-4,36,36.5,32.5,4.0,0.15,W D D W W,39133,Dominic Calvert-Lewin - 11,Jordan Pickford,NaN,1.250000,0.286505


## xG-adjusted points

Next, we determine for each player their 'adjusted points'. To do this, we first subtract for each player all the points they have accumulated through goals, assists and clean sheets. Then, we add points for each player based on their expected goals, assists and clean sheets. This gives a much improved estimate of each player's true point generating capability. 

In [7]:
def incorporate_xG(indicator, ix):
    xG = playerStats.loc[indicator, 'xG'].values[0]  
    penalty_attempts = playerStats.loc[indicator,'PKatt'].values[0]
    if df.loc[ix, 'element_type']<=2:
        # 0.24 is the probability to miss a penalty, which incurs -2 points
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] - 6 * (df.loc[ix, 'goals_scored'] - xG) \
                                                                    - 2 * 0.24 * penalty_attempts
    elif df.loc[ix, 'element_type']==3:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] -  5 * (df.loc[ix, 'goals_scored'] - xG) \
                                                                    - 2 * 0.24 * penalty_attempts
    elif df.loc[ix, 'element_type']==4:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] -  4 * (df.loc[ix, 'goals_scored'] - xG) \
                                                                    - 2 * 0.24 * penalty_attempts

In [8]:
# always run 'team_xGA' AFTER 'incorporate_xG'
def team_xGA(indicator, ix):
    team = team_names[df.loc[ix, 'team']-1]
    clean_sheets = df.loc[ix, 'clean_sheets']
    probability_cleanSheet = teamStats.loc[teamStats['Squad']==team, 'probability no goals allowed'].values[0]
    if df.loc[ix, 'element_type']<=2:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  \
                    4 * (df.loc[ix, 'clean_sheets'] - df.loc[ix, 'games played']*probability_cleanSheet)
    elif df.loc[ix, 'element_type']==3:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  \
                    (df.loc[ix, 'clean_sheets'] - df.loc[ix, 'games played']*probability_cleanSheet)

A player who gives an assist that directly leads to a shot, is assigned the xG-value of the shot in xA (expected assists), i.e. xA is a measure of 'goal assists'. In FPL, however, the definition of an assist is somewhat more relaxed, e.g. goals resulting from a rebound of parried shot will award an assist to the player making the initial shot. For this reason, we calculate the total number of assists awarded in FPL and the total sum of xA of all players and get an estimate of the proportion of assists that xA covers in FPL. Then, we modify that proportion of players assists based on their xA.

In [9]:
# always run 'xA' AFTER 'incorporate_xG'
def xA(indicator, ix):
    xA = playerStats.loc[indicator, 'xA'].values[0]
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  3 * (xA_proportion*df.loc[ix, 'assists'] - xA)

In [10]:
def calculateAdjustedPoints(indicator, index):
    incorporate_xG(indicator, index)
    team_xGA(indicator, index)
    xA(indicator, index)

In [11]:
team_names = np.sort(playerStats['Squad'].unique())
xA_proportion = playerStats['xA'].sum()/df['assists'].sum()
df['points_per_game'] = df['points_per_game'].astype(float)
df['games played'] = df['total_points']/df['points_per_game']
#df['games played'] = df['minutes']/90.0
xA_proportion

0.7631410256410257

## Main loop for assigning adjusted points

Below is the main loop where we calculate adjusted points for each player. Calculation of the adjusted points itself is straightforward, but there is some work required to match players in two different data sets. Comparing player names in both data sets gives unique matches in many cases, but some special cases need to be covered through individual solutions.

In [12]:
for ix in df[df['minutes']>0].index: 
    # player name in FPL data
    name = df.loc[ix, 'web_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u').replace('ö', 'o').\
                        replace('ä', 'a')
    # find FBREF data indexes where player name contains 'name'
    indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains(name)
    # if unique match is found, we can calculate 'adjusted points'
    if playerStats.loc[indicator].shape[0]==1:
        calculateAdjustedPoints(indicator, ix)
    # if no match is found
    elif playerStats.loc[indicator].shape[0]==0:
        # try matching based on first name, helps in some cases
        first_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u')\
                                                                    .replace('ö', 'o').replace('ä', 'a')
        first_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(first_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[first_name_indicator].shape[0]==1:
            calculateAdjustedPoints(first_name_indicator, ix)
        # here we deal with some special cases individually
        else:
            names = ['rodrigo','garcia','chicharito','elliott']
            if name == 'rodrigo':
                exceptional_case_indicator = \
                (playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodri')) & \
                (~playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodriguez'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'chicharito':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('javierhernandez'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'elliott':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('elliot'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            else:
                print(str(ix) + ': no player found.')
    # if more than one matches were found in the original comparison
    elif playerStats.loc[indicator].shape[0]>1:
        # try matching full name
        full_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '') \
            + df.loc[ix, 'second_name'].lower().replace(' ', '').replace('-', '')
        full_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(full_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[full_name_indicator].shape[0]==1:
            calculateAdjustedPoints(full_name_indicator, ix)
        # here we try to match players based both on their name and team
        else:
            team = team_names[df.loc[ix, 'team']-1]
            team_indicator = playerStats['Squad']==team
            if playerStats.loc[indicator & team_indicator].shape[0]==1:
                calculateAdjustedPoints(indicator & team_indicator, ix)
            # special case
            else:
                if name=='son':
                    exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('heung')
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': non-unique name.')
    else:
        print(str(ix) + 'Player not found')

In [13]:
df['adjusted points per game'] = df['adjusted points'] / df['games played']

In [14]:
# give a sorted list showing the players with highest 'adjusted points per game'
df[['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game']]\
                                .sort_values(by='adjusted points per game', ascending=False)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game
id,,,,,,
191,Salah,23.013699,168,7.3,162.287922,7.051797
281,McGovern,2.000000,11,5.5,12.627644,6.313822
215,De Bruyne,25.000000,175,7.0,156.276740,6.251070
192,Mané,23.030303,152,6.6,139.255400,6.046616
233,Rashford,21.967213,134,6.1,125.692885,5.721840
182,Alexander-Arnold,25.932203,153,5.9,144.130662,5.557980
214,Sterling,23.200000,116,5.0,128.701291,5.547469
166,Vardy,24.062500,154,6.4,131.743462,5.475053
210,Agüero,19.838710,123,6.2,104.182308,5.251466


In [15]:
# save data
filepath = '../data/data_week' + str(latest_gameweek) + str('.csv')
df.to_csv(filepath)

filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats.to_csv(filepath)

Below we check how well the total xG matches the total scored goals.

In [16]:
total_xG = playerStats['xG'].sum()
total_goals = df['goals_scored'].sum()
print('Total xG: ' + str(total_xG))
print('Total goals: ' + str(total_goals))
print('Total xG / total goals: ' + str(total_xG/total_goals))

Total xG: 706.4
Total goals: 693
Total xG / total goals: 1.0193362193362192
